In [1]:
import pandas as pd
import numpy as np
#pd.set_option("display.max_columns", None)
#pd.set_option('display.max_rows', None)

In [2]:
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
pio.templates.default = "plotly_dark"

In [3]:
circuits              = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/circuits.csv')
constructor_results   = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/constructor_results.csv')
constructor_standings = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/constructor_standings.csv')
constructors          = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/constructors.csv')
driver_standings      = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/driver_standings.csv')
drivers               = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/drivers.csv')
lap_times             = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/lap_times.csv')
pit_stops             = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/pit_stops.csv')
qualifying            = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/qualifying.csv')
races                 = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/races.csv')
results               = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/results.csv')
seasons               = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/seasons.csv')
status                = pd.read_csv('https://raw.githubusercontent.com/frax0n/F1-Analysis/Frax/datasets/status.csv')

Just considering the turbohybrid Era

In [4]:
TH_races=races.loc[races['year']>=2014]

Loading QualyData and separating Q1,Q2 and Q3 data

In [5]:
qualifying=qualifying.rename(columns={"position":"qualifying_pos"})
qualifying=qualifying.replace("\\N",np.nan)

In [6]:
#Default Format was String , changing it to Time 
qualifying['q1']=pd.to_datetime(qualifying['q1'],format="%M:%S.%f")
qualifying['q2']=pd.to_datetime(qualifying['q2'],format="%M:%S.%f")
qualifying['q3']=pd.to_datetime(qualifying['q3'],format="%M:%S.%f")
qualifying['q1_s']=qualifying['q1'].dt.minute*60+qualifying['q1'].dt.second+qualifying['q1'].dt.microsecond*0.000001
qualifying['q2_s']=qualifying['q2'].dt.minute*60+qualifying['q2'].dt.second+qualifying['q2'].dt.microsecond*0.000001
qualifying['q3_s']=qualifying['q3'].dt.minute*60+qualifying['q3'].dt.second+qualifying['q3'].dt.microsecond*0.000001
qualifying.drop(columns=['q1', 'q2', 'q3'],inplace=True)

In [7]:
#Getting Deltas instead of times as the times are constantly changing accd to the cars of that year
qualifying['q1min']=qualifying['q1_s']
qualifying['q2min']=qualifying['q2_s']
qualifying['q3min']=qualifying['q3_s']
qualifying=qualifying.merge(qualifying.groupby('raceId').aggregate({'q1min':'min','q2min':'min','q3min':'min'}),left_on='raceId',right_on='raceId')
qualifying=qualifying.rename(columns={"q1min_y":"q1min","q2min_y":"q2min","q3min_y":"q3min"})
qualifying.drop(columns=['q1min_x','q2min_x','q3min_x'],inplace=True)
qualifying['q1_delta']=qualifying['q1min']
qualifying['q2_delta']=qualifying['q2min']
qualifying['q3_delta']=qualifying['q3min']
qualifying['q1_delta']=qualifying['q1_s']-qualifying['q1min']
qualifying['q2_delta']=qualifying['q2_s']-qualifying['q2min']
qualifying['q3_delta']=qualifying['q3_s']-qualifying['q3min']
qualifyingq1=qualifying[['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos','q1_s','q1_delta']]
qualifyingq2=qualifying[['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos','q2_s','q2_delta']]
qualifyingq3=qualifying[['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos','q3_s','q3_delta']]


In [8]:
#Getting Results for the final positions for the GrandPrix
results_temp=results[['resultId', 'raceId', 'driverId', 'constructorId','position','statusId','rank']]
results_temp.rename(columns={'rank':'driverRanking','position':'final_position',},inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [9]:
#Getting Drivers , Constructors and other data for merging

In [10]:
TH_races_temp=TH_races[['raceId','year','circuitId']]
constructor_standings_temp=constructor_standings[['constructorStandingsId', 'raceId', 'constructorId','position']]
constructor_standings_temp.rename(columns={'position':'constructor_ranking'},inplace=True)
pit_stops_temp=pit_stops.drop(columns=['time','duration'])

In [11]:
temp=qualifyingq2
temp=temp.dropna()
qualifyingq2=temp
temp=qualifyingq3
temp=temp.dropna()
qualifyingq3=temp

## TEST 

In [12]:
qualifyingq1 = qualifyingq1.loc[qualifyingq1['raceId']>=900]

In [13]:
qualifyingq1

,qualifyId,raceId,driverId,constructorId,number,qualifying_pos,q1_s,q1_delta
5858,5860,900,1,131,44,1,91.699,0.924
5859,5861,900,817,9,3,2,90.775,0.000
5860,5862,900,3,131,6,3,92.564,1.789
5861,5863,900,825,1,20,4,90.949,0.174
5862,5864,900,4,6,14,5,91.388,0.613
...,...,...,...,...,...,...,...,...
8769,8810,1055,852,213,22,16,78.556,0.735
8770,8811,1055,8,51,7,17,78.917,1.096
8771,8812,1055,854,210,47,18,79.117,1.296
8772,8813,1055,849,3,6,19,79.219,1.398


In [14]:
qualifyingq1['Stop 1'] = np.nan
qualifyingq1['Stop 2'] = np.nan
qualifyingq1['Stop 1 Lap'] = np.nan
qualifyingq1['Stop 2 Lap'] = np.nan
qualifyingq1['Stop 1 Time'] = np.nan
qualifyingq1['Stop 2 Time'] = np.nan

In [15]:
qualifyingq2['Stop 1'] = np.nan
qualifyingq2['Stop 2'] = np.nan
qualifyingq2['Stop 1 Lap'] = np.nan
qualifyingq2['Stop 2 Lap'] = np.nan
qualifyingq2['Stop 1 Time'] = np.nan
qualifyingq2['Stop 2 Time'] = np.nan

In [16]:
qualifyingq3['Stop 1'] = np.nan
qualifyingq3['Stop 2'] = np.nan
qualifyingq3['Stop 1 Lap'] = np.nan
qualifyingq3['Stop 2 Lap'] = np.nan
qualifyingq3['Stop 1 Time'] = np.nan
qualifyingq3['Stop 2 Time'] = np.nan

In [17]:
qualifyingq1

,qualifyId,raceId,driverId,constructorId,number,qualifying_pos,q1_s,q1_delta,Stop 1,Stop 2,Stop 1 Lap,Stop 2 Lap,Stop 1 Time,Stop 2 Time
5858,5860,900,1,131,44,1,91.699,0.924,NaN,NaN,NaN,NaN,NaN,NaN
5859,5861,900,817,9,3,2,90.775,0.000,NaN,NaN,NaN,NaN,NaN,NaN
5860,5862,900,3,131,6,3,92.564,1.789,NaN,NaN,NaN,NaN,NaN,NaN
5861,5863,900,825,1,20,4,90.949,0.174,NaN,NaN,NaN,NaN,NaN,NaN
5862,5864,900,4,6,14,5,91.388,0.613,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8769,8810,1055,852,213,22,16,78.556,0.735,NaN,NaN,NaN,NaN,NaN,NaN
8770,8811,1055,8,51,7,17,78.917,1.096,NaN,NaN,NaN,NaN,NaN,NaN
8771,8812,1055,854,210,47,18,79.117,1.296,NaN,NaN,NaN,NaN,NaN,NaN
8772,8813,1055,849,3,6,19,79.219,1.398,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
for i in range(qualifyingq1.shape[0]):
    t = pit_stops.loc[(pit_stops['raceId']==qualifyingq1['raceId'].iloc[i]) & (pit_stops['driverId']==qualifyingq1['driverId'].iloc[i])]
    if t.shape[0]==1:
        qualifyingq1['Stop 1'].iloc[i]=1
        qualifyingq1['Stop 2'].iloc[i]=0
        qualifyingq1['Stop 1 Lap'].iloc[i] = t['lap'].iloc[0]
        qualifyingq1['Stop 2 Lap'].iloc[i]=0
        qualifyingq1['Stop 1 Time'].iloc[i]= t['milliseconds'].iloc[0]*0.001
        qualifyingq1['Stop 2 Time'].iloc[i]=0
        
    if t.shape[0]==2:
        qualifyingq1['Stop 1'].iloc[i]=1
        qualifyingq1['Stop 2'].iloc[i]=1
        qualifyingq1['Stop 1 Lap'].iloc[i] = t['lap'].iloc[0]
        qualifyingq1['Stop 2 Lap'].iloc[i]=  t['lap'].iloc[1]
        qualifyingq1['Stop 1 Time'].iloc[i]= t['milliseconds'].iloc[0]*0.001
        qualifyingq1['Stop 2 Time'].iloc[i]= t['milliseconds'].iloc[1]*0.001
    
    if t.shape[0]>2:
        qualifyingq1['Stop 1'].iloc[i]=np.nan
        qualifyingq1['Stop 2'].iloc[i]=np.nan
        qualifyingq1['Stop 1 Lap'].iloc[i] = np.nan
        qualifyingq1['Stop 2 Lap'].iloc[i]= np.nan
        qualifyingq1['Stop 1 Time'].iloc[i]= np.nan
        qualifyingq1['Stop 2 Time'].iloc[i]= np.nan

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

...

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [19]:
for i in range(qualifyingq2.shape[0]):
    t = pit_stops.loc[(pit_stops['raceId']==qualifyingq2['raceId'].iloc[i]) & (pit_stops['driverId']==qualifyingq2['driverId'].iloc[i])]
    if t.shape[0]==1:
        qualifyingq2['Stop 1'].iloc[i]=1
        qualifyingq2['Stop 2'].iloc[i]=0
        qualifyingq2['Stop 1 Lap'].iloc[i] = t['lap'].iloc[0]
        qualifyingq2['Stop 2 Lap'].iloc[i]=0
        qualifyingq2['Stop 1 Time'].iloc[i]= t['milliseconds'].iloc[0]*0.001
        qualifyingq2['Stop 2 Time'].iloc[i]=0
        
    if t.shape[0]==2:
        qualifyingq2['Stop 1'].iloc[i]=1
        qualifyingq2['Stop 2'].iloc[i]=1
        qualifyingq2['Stop 1 Lap'].iloc[i] = t['lap'].iloc[0]
        qualifyingq2['Stop 2 Lap'].iloc[i]=  t['lap'].iloc[1]
        qualifyingq2['Stop 1 Time'].iloc[i]= t['milliseconds'].iloc[0]*0.001
        qualifyingq2['Stop 2 Time'].iloc[i]= t['milliseconds'].iloc[1]*0.001
    
    if t.shape[0]>2:
        qualifyingq2['Stop 1'].iloc[i]=np.nan
        qualifyingq2['Stop 2'].iloc[i]=np.nan
        qualifyingq2['Stop 1 Lap'].iloc[i]=np.nan
        qualifyingq2['Stop 2 Lap'].iloc[i]=np.nan
        qualifyingq2['Stop 1 Time'].iloc[i]=np.nan
        qualifyingq2['Stop 2 Time'].iloc[i]=np.nan

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

...

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [20]:
for i in range(qualifyingq3.shape[0]):
    t = pit_stops.loc[(pit_stops['raceId']==qualifyingq3['raceId'].iloc[i]) & (pit_stops['driverId']==qualifyingq3['driverId'].iloc[i])]
    if t.shape[0]==1:
        qualifyingq3['Stop 1'].iloc[i]=1
        qualifyingq3['Stop 2'].iloc[i]=0
        qualifyingq3['Stop 1 Lap'].iloc[i] = t['lap'].iloc[0]
        qualifyingq3['Stop 2 Lap'].iloc[i]=0
        qualifyingq3['Stop 1 Time'].iloc[i]= t['milliseconds'].iloc[0]*0.001
        qualifyingq3['Stop 2 Time'].iloc[i]=0
        
    if t.shape[0]==2:
        qualifyingq3['Stop 1'].iloc[i]=1
        qualifyingq3['Stop 2'].iloc[i]=1
        qualifyingq3['Stop 1 Lap'].iloc[i] = t['lap'].iloc[0]
        qualifyingq3['Stop 2 Lap'].iloc[i]=  t['lap'].iloc[1]
        qualifyingq3['Stop 1 Time'].iloc[i]= t['milliseconds'].iloc[0]*0.001
        qualifyingq3['Stop 2 Time'].iloc[i]= t['milliseconds'].iloc[1]*0.001
    
    if t.shape[0]>2:
        qualifyingq3['Stop 1'].iloc[i]=np.nan
        qualifyingq3['Stop 2'].iloc[i]=np.nan
        qualifyingq3['Stop 1 Lap'].iloc[i]=np.nan
        qualifyingq3['Stop 2 Lap'].iloc[i]=np.nan
        qualifyingq3['Stop 1 Time'].iloc[i]=np.nan
        qualifyingq3['Stop 2 Time'].iloc[i]=np.nan

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

...

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

In [21]:
qualifyingq1 = qualifyingq1.dropna()
qualifyingq2 = qualifyingq2.dropna()
qualifyingq3 = qualifyingq3.dropna()

In [22]:
qualifyingq2

,qualifyId,raceId,driverId,constructorId,number,qualifying_pos,q2_s,q2_delta,Stop 1,Stop 2,Stop 1 Lap,Stop 2 Lap,Stop 1 Time,Stop 2 Time
4536,4537,841,20,9,1,1,84.090,0.000,1.0,1.0,14.0,36.0,22.603,24.036
4537,4538,841,1,1,3,2,84.595,0.505,1.0,1.0,16.0,36.0,23.227,23.199
4541,4542,841,808,4,10,6,85.582,1.492,1.0,1.0,16.0,36.0,24.535,25.683
4542,4543,841,3,131,8,7,85.606,1.516,1.0,0.0,16.0,0.0,23.716,0.000
4544,4545,841,155,15,16,9,85.405,1.315,1.0,1.0,16.0,32.0,24.064,24.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8764,8805,1055,840,117,18,11,77.974,1.052,1.0,1.0,22.0,39.0,21.756,21.432
8765,8806,1055,842,213,10,12,77.982,1.060,1.0,1.0,18.0,47.0,28.042,26.065
8766,8807,1055,20,117,5,13,78.079,1.157,1.0,1.0,21.0,38.0,23.087,21.973
8767,8808,1055,841,51,99,14,78.356,1.434,1.0,1.0,8.0,39.0,54.673,22.871


In [23]:
t = pit_stops.loc[(pit_stops['raceId']==qualifyingq1['raceId'].iloc[120]) & (pit_stops['driverId']==qualifyingq1['driverId'].iloc[120])]
t.shape[0]

2

## WORKING

In [24]:
q1=qualifyingq1.merge(results_temp,left_on=['raceId','driverId','constructorId'],right_on=['raceId','driverId','constructorId'])
q1=q1.merge(constructor_standings_temp,left_on=['raceId','constructorId'],right_on=['raceId','constructorId'])
q1=q1.merge(TH_races_temp,left_on=['raceId'],right_on=['raceId'])
#q1=q1.merge(pit_stops_temp,left_on=['raceId','driverId'],right_on=['raceId','driverId'])
q1=q1.loc[(q1['statusId']==1) | (q1['statusId']==11) | (q1['statusId']==12)]

In [25]:
q2=qualifyingq2.merge(results_temp,left_on=['raceId','driverId','constructorId'],right_on=['raceId','driverId','constructorId'])
q2=q2.merge(constructor_standings_temp,left_on=['raceId','constructorId'],right_on=['raceId','constructorId'])
q2=q2.merge(TH_races_temp,left_on=['raceId'],right_on=['raceId'])
#q2=q2.merge(pit_stops_temp,left_on=['raceId','driverId'],right_on=['raceId','driverId'])
q2=q2.loc[(q2['statusId']==1) | (q2['statusId']==11) | (q2['statusId']==12)]

In [26]:
q3=qualifyingq3.merge(results_temp,left_on=['raceId','driverId','constructorId'],right_on=['raceId','driverId','constructorId'])
q3=q3.merge(constructor_standings_temp,left_on=['raceId','constructorId'],right_on=['raceId','constructorId'])
q3=q3.merge(TH_races_temp,left_on=['raceId'],right_on=['raceId'])
#q3=q3.merge(pit_stops_temp,left_on=['raceId','driverId'],right_on=['raceId','driverId'])
q3=q3.loc[(q3['statusId']==1) | (q3['statusId']==11) | (q3['statusId']==12)]

In [27]:
q1

,qualifyId,raceId,driverId,constructorId,number,qualifying_pos,q1_s,q1_delta,Stop 1,Stop 2,...,Stop 1 Time,Stop 2 Time,resultId,final_position,statusId,driverRanking,constructorStandingsId,constructor_ranking,year,circuitId
1,5862,900,3,131,6,3,92.564,1.789,1.0,1.0,...,24.331,23.673,22130,1,1,1,25931,2,2014,1
2,5863,900,825,1,20,4,90.949,0.174,1.0,1.0,...,23.150,22.273,22131,2,1,6,25932,1,2014,1
3,5870,900,18,1,22,11,91.396,0.621,1.0,1.0,...,22.411,22.399,22132,3,1,5,25932,1,2014,1
4,5864,900,4,6,14,5,91.388,0.613,1.0,1.0,...,22.887,21.978,22133,4,1,7,25933,3,2014,1
5,5871,900,8,6,7,12,92.439,1.664,1.0,1.0,...,25.543,21.825,22136,7,1,8,25933,3,2014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,8808,1055,841,51,99,14,78.549,0.728,1.0,1.0,...,54.673,22.871,25040,15,11,13,27729,8,2021,4
2036,8811,1055,8,51,7,17,78.917,1.096,1.0,0.0,...,21.572,0.000,25037,12,11,15,27729,8,2021,4
2037,8809,1055,847,3,63,15,78.445,0.624,1.0,1.0,...,23.814,21.919,25039,14,11,17,27731,9,2021,4
2038,8812,1055,854,210,47,18,79.117,1.296,1.0,1.0,...,24.006,22.310,25043,18,12,12,27732,10,2021,4


# Machine Learning

In [28]:
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble  import RandomForestClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score

from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn import svm

In [29]:
# q1.drop(columns=['qualifyId','resultId','constructorStandingsId','raceId','q1_s','year'],inplace=True)
# q2.drop(columns=['qualifyId','resultId','constructorStandingsId','raceId','q2_s','year'],inplace=True)
# q3.drop(columns=['qualifyId','resultId','constructorStandingsId','raceId','q3_s','year'],inplace=True)

In [30]:
q1[['Stop 1', 'Stop 2', 'Stop 1 Lap', 'Stop 2 Lap']] = q1[['Stop 1', 'Stop 2', 'Stop 1 Lap', 'Stop 2 Lap']].astype(int)
q2[['Stop 1', 'Stop 2', 'Stop 1 Lap', 'Stop 2 Lap']] = q2[['Stop 1', 'Stop 2', 'Stop 1 Lap', 'Stop 2 Lap']].astype(int)
q2[['Stop 1', 'Stop 2', 'Stop 1 Lap', 'Stop 2 Lap']] = q2[['Stop 1', 'Stop 2', 'Stop 1 Lap', 'Stop 2 Lap']].astype(int)

In [31]:
q1['final_position']=q1['final_position'].astype(int)
q1['driverRanking']=q1['driverRanking'].astype(int)
q2['final_position']=q2['final_position'].astype(int)
q2['driverRanking']=q2['driverRanking'].astype(int)
q3['final_position']=q3['final_position'].astype(int)
q3['driverRanking']=q3['driverRanking'].astype(int)

In [32]:
q1.columns

Index(['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos', 'q1_s', 'q1_delta', 'Stop 1', 'Stop 2', 'Stop 1 Lap',
       'Stop 2 Lap', 'Stop 1 Time', 'Stop 2 Time', 'resultId',
       'final_position', 'statusId', 'driverRanking', 'constructorStandingsId',
       'constructor_ranking', 'year', 'circuitId'],
      dtype='object')

In [33]:
q1

,qualifyId,raceId,driverId,constructorId,number,qualifying_pos,q1_s,q1_delta,Stop 1,Stop 2,...,Stop 1 Time,Stop 2 Time,resultId,final_position,statusId,driverRanking,constructorStandingsId,constructor_ranking,year,circuitId
1,5862,900,3,131,6,3,92.564,1.789,1,1,...,24.331,23.673,22130,1,1,1,25931,2,2014,1
2,5863,900,825,1,20,4,90.949,0.174,1,1,...,23.150,22.273,22131,2,1,6,25932,1,2014,1
3,5870,900,18,1,22,11,91.396,0.621,1,1,...,22.411,22.399,22132,3,1,5,25932,1,2014,1
4,5864,900,4,6,14,5,91.388,0.613,1,1,...,22.887,21.978,22133,4,1,7,25933,3,2014,1
5,5871,900,8,6,7,12,92.439,1.664,1,1,...,25.543,21.825,22136,7,1,8,25933,3,2014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,8808,1055,841,51,99,14,78.549,0.728,1,1,...,54.673,22.871,25040,15,11,13,27729,8,2021,4
2036,8811,1055,8,51,7,17,78.917,1.096,1,0,...,21.572,0.000,25037,12,11,15,27729,8,2021,4
2037,8809,1055,847,3,63,15,78.445,0.624,1,1,...,23.814,21.919,25039,14,11,17,27731,9,2021,4
2038,8812,1055,854,210,47,18,79.117,1.296,1,1,...,24.006,22.310,25043,18,12,12,27732,10,2021,4


In [34]:
test1=q1

In [35]:
test1.shape

(1756, 22)

In [36]:
test2=q2.merge(q1,
               left_on=['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos',  'Stop 1', 'Stop 2', 'Stop 1 Lap',
       'Stop 2 Lap', 'Stop 1 Time', 'Stop 2 Time', 'resultId',
       'final_position', 'statusId', 'driverRanking', 'constructorStandingsId',
       'constructor_ranking', 'year', 'circuitId'],
               right_on=['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos',  'Stop 1', 'Stop 2', 'Stop 1 Lap',
       'Stop 2 Lap', 'Stop 1 Time', 'Stop 2 Time', 'resultId',
       'final_position', 'statusId', 'driverRanking', 'constructorStandingsId',
       'constructor_ranking', 'year', 'circuitId'])
test3=q3.merge(test2,
              left_on=['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos',  'Stop 1', 'Stop 2', 'Stop 1 Lap',
       'Stop 2 Lap', 'Stop 1 Time', 'Stop 2 Time', 'resultId',
       'final_position', 'statusId', 'driverRanking', 'constructorStandingsId',
       'constructor_ranking', 'year', 'circuitId'],
              right_on=['qualifyId', 'raceId', 'driverId', 'constructorId', 'number',
       'qualifying_pos',  'Stop 1', 'Stop 2', 'Stop 1 Lap',
       'Stop 2 Lap', 'Stop 1 Time', 'Stop 2 Time', 'resultId',
       'final_position', 'statusId', 'driverRanking', 'constructorStandingsId',
       'constructor_ranking', 'year', 'circuitId'])

In [37]:
test1.drop(columns=['qualifyId','resultId','constructorStandingsId','raceId','q1_s','year'],inplace=True)
test2.drop(columns=['qualifyId','resultId','constructorStandingsId','raceId','q2_s','year'],inplace=True)
test3.drop(columns=['qualifyId','resultId','constructorStandingsId','raceId','q3_s','year'],inplace=True)

In [38]:
test1

,driverId,constructorId,number,qualifying_pos,q1_delta,Stop 1,Stop 2,Stop 1 Lap,Stop 2 Lap,Stop 1 Time,Stop 2 Time,final_position,statusId,driverRanking,constructor_ranking,circuitId
1,3,131,6,3,1.789,1,1,12,38,24.331,23.673,1,1,1,2,1
2,825,1,20,4,0.174,1,1,12,37,23.150,22.273,2,1,6,1,1
3,18,1,22,11,0.621,1,1,11,32,22.411,22.399,3,1,5,1,1
4,4,6,14,5,0.613,1,1,12,35,22.887,21.978,4,1,7,3,1
5,8,6,7,12,1.664,1,1,12,36,25.543,21.825,7,1,8,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,841,51,99,14,0.728,1,1,8,39,54.673,22.871,15,11,13,8,4
2036,8,51,7,17,1.096,1,0,37,0,21.572,0.000,12,11,15,8,4
2037,847,3,63,15,0.624,1,1,9,28,23.814,21.919,14,11,17,9,4
2038,854,210,47,18,1.296,1,1,23,50,24.006,22.310,18,12,12,10,4


In [39]:
X_train  , X_test  , y_train  , y_test = train_test_split(test1.drop(columns=['final_position','qualifying_pos','number','statusId']), test1['final_position'], test_size = 0.25, random_state=8)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(test2.drop(columns=['final_position','qualifying_pos','number','statusId','q1_s']), test2['final_position'], test_size = 0.25, random_state=8)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(test3.drop(columns=['final_position','qualifying_pos','number','statusId','q2_s','q1_s']), test3['final_position'], test_size = 0.25, random_state=8)

In [40]:
X_train = pd.get_dummies(X_train,columns=['driverId'])
X_train = pd.get_dummies(X_train,columns=['circuitId'])
X_train = pd.get_dummies(X_train,columns=['constructorId'])
#X_train.drop(columns=['driverId','constructorId','circuitId'])
X_test = pd.get_dummies(X_test,columns=['driverId'])
X_test = pd.get_dummies(X_test,columns=['circuitId'])
X_test = pd.get_dummies(X_test,columns=['constructorId'])
#X_test.drop(columns=['driverId','constructorId','circuitId'])
X_test = X_test.reindex(columns = X_train.columns, fill_value=0)

In [41]:
X_train_2 = pd.get_dummies(X_train_2,columns=['driverId'])
X_train_2= pd.get_dummies(X_train_2,columns=['circuitId'])
X_train_2= pd.get_dummies(X_train_2,columns=['constructorId'])
#X_train_3.drop(columns=['driverId','constructorId','circuitId'])

In [42]:
X_train_3 = pd.get_dummies(X_train_3,columns=['driverId'])
X_train_3= pd.get_dummies(X_train_3,columns=['circuitId'])
X_train_3= pd.get_dummies(X_train_3,columns=['constructorId'])
#X_train_3.drop(columns=['driverId','constructorId','circuitId'])

In [43]:
X_train

,q1_delta,Stop 1,Stop 2,Stop 1 Lap,Stop 2 Lap,Stop 1 Time,Stop 2 Time,driverRanking,constructor_ranking,driverId_1,...,constructorId_117,constructorId_131,constructorId_206,constructorId_207,constructorId_208,constructorId_209,constructorId_210,constructorId_211,constructorId_213,constructorId_214
362,0.745,1,1,21,44,28.781,31.937,5,3,0,...,0,0,0,0,0,0,0,0,0,0
1773,0.774,1,1,3,29,28.286,21.880,14,5,0,...,0,0,0,0,0,0,0,0,0,0
786,1.181,1,0,11,0,26.542,0.000,17,9,0,...,0,0,0,0,0,0,0,0,0,0
1583,1.095,1,0,28,0,23.676,0.000,7,5,0,...,0,0,0,0,0,0,0,0,0,0
1292,0.000,1,0,24,0,22.614,0.000,2,1,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,0.868,1,1,15,34,23.257,26.335,10,4,0,...,0,0,0,0,0,0,0,0,0,0
172,0.130,1,0,24,0,24.583,0.000,2,1,0,...,0,1,0,0,0,0,0,0,0,0
445,0.176,1,1,19,38,24.640,25.242,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1594,2.312,1,1,11,31,23.475,23.000,9,6,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
X_train

,q1_delta,Stop 1,Stop 2,Stop 1 Lap,Stop 2 Lap,Stop 1 Time,Stop 2 Time,driverRanking,constructor_ranking,driverId_1,...,constructorId_117,constructorId_131,constructorId_206,constructorId_207,constructorId_208,constructorId_209,constructorId_210,constructorId_211,constructorId_213,constructorId_214
362,0.745,1,1,21,44,28.781,31.937,5,3,0,...,0,0,0,0,0,0,0,0,0,0
1773,0.774,1,1,3,29,28.286,21.880,14,5,0,...,0,0,0,0,0,0,0,0,0,0
786,1.181,1,0,11,0,26.542,0.000,17,9,0,...,0,0,0,0,0,0,0,0,0,0
1583,1.095,1,0,28,0,23.676,0.000,7,5,0,...,0,0,0,0,0,0,0,0,0,0
1292,0.000,1,0,24,0,22.614,0.000,2,1,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,0.868,1,1,15,34,23.257,26.335,10,4,0,...,0,0,0,0,0,0,0,0,0,0
172,0.130,1,0,24,0,24.583,0.000,2,1,0,...,0,1,0,0,0,0,0,0,0,0
445,0.176,1,1,19,38,24.640,25.242,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1594,2.312,1,1,11,31,23.475,23.000,9,6,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
X_train_2

,q2_delta,Stop 1,Stop 2,Stop 1 Lap,Stop 2 Lap,Stop 1 Time,Stop 2 Time,driverRanking,constructor_ranking,q1_delta,...,constructorId_10,constructorId_15,constructorId_51,constructorId_117,constructorId_131,constructorId_206,constructorId_208,constructorId_210,constructorId_211,constructorId_213
346,1.918,1,1,10,38,22.874,22.937,11,5,1.494,...,0,0,0,0,0,0,0,0,0,0
682,1.117,1,0,22,0,23.868,0.000,12,5,0.705,...,0,0,0,0,0,0,0,0,0,0
428,1.424,1,1,15,28,24.537,24.033,12,5,1.585,...,1,0,0,0,0,0,0,0,0,0
166,4.092,1,1,17,35,25.268,35.776,14,7,1.767,...,1,0,0,0,0,0,0,0,0,0
8,1.588,1,1,10,36,34.921,23.117,3,4,0.826,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0.000,1,1,15,34,24.433,24.513,5,1,1.107,...,0,0,0,0,1,0,0,0,0,0
986,0.186,1,0,7,0,23.485,0.000,10,3,0.823,...,0,0,0,0,0,0,0,0,0,0
133,2.198,1,0,39,0,30.303,0.000,18,10,2.400,...,0,1,0,0,0,0,0,0,0,0
361,1.342,1,1,20,32,24.934,25.182,9,4,1.100,...,0,0,0,0,0,0,0,0,0,0


In [46]:
X_train_3

,q3_delta,Stop 1,Stop 2,Stop 1 Lap,Stop 2 Lap,Stop 1 Time,Stop 2 Time,driverRanking,constructor_ranking,q2_delta,...,constructorId_10,constructorId_15,constructorId_51,constructorId_117,constructorId_131,constructorId_208,constructorId_210,constructorId_211,constructorId_213,constructorId_214
730,0.000,1.0,0.0,21.0,0.0,22.449,0.000,4,2,0.000,...,0,0,0,0,0,0,0,0,0,0
286,1.246,1.0,1.0,15.0,44.0,22.099,21.897,8,7,1.156,...,0,0,0,0,0,0,0,0,0,0
794,1.657,1.0,0.0,8.0,0.0,22.974,0.000,12,4,1.221,...,0,0,0,0,0,0,0,0,0,0
837,1.005,1.0,1.0,27.0,42.0,22.351,22.388,8,3,0.653,...,0,0,0,0,0,0,0,1,0,0
580,0.687,1.0,0.0,16.0,0.0,28.542,0.000,8,1,0.060,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,1.455,1.0,1.0,14.0,42.0,22.859,22.178,9,7,1.309,...,0,0,0,0,0,0,0,0,0,0
133,0.777,1.0,1.0,14.0,40.0,21.762,21.836,6,2,1.001,...,0,0,0,0,0,0,0,0,0,0
361,0.000,1.0,0.0,34.0,0.0,30.097,0.000,2,2,0.774,...,0,0,0,0,0,0,0,0,0,0
340,1.746,1.0,1.0,8.0,26.0,21.834,21.482,13,4,1.327,...,1,0,0,0,0,0,0,0,0,0


## HyperOpt

In [40]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe
from hpsklearn import random_forest,svc,knn,extra_trees

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [41]:
model = HyperoptEstimator(classifier=any_classifier('clf'), preprocessing=any_preprocessing('pre'), algo=tpe.suggest, trial_timeout=30)

In [42]:
X_train = np.asarray(X_train).astype(np.float32)
y_train=np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_test=np.asarray(y_test).astype(np.float32)

In [43]:
model.fit(X_train, y_train)
# summarize performance
acc = model.score(X_test, y_test)
print("Accuracy: %.3f" % acc)
# summarize the best model
print(model.best_model())

100%|████████████████████████████████████████████████████████████████████| 1/1 [00:34<00:00, 34.04s/trial, best loss=?]


AllTrialsFailed: 

In [43]:
y4 = model.predict(X_test)
acc = accuracy_score(y_test, y4)
print("Accuracy: %.3f" % acc)

RuntimeError: Attempting to use a model that has not been fit. Ensure fit() has been called and at least one trial has completed without failing or timing out.

In [ ]:
difference=y_test-y4
sns.kdeplot(difference)

In [ ]:
from sklearn.metrics import confusion_matrix  
import seaborn as sn
fig,axes=plt.subplots(1,2,figsize=(15,6))
cm = pd.DataFrame(confusion_matrix(y_test, y4))
sn.heatmap(cm,ax=axes[0])
cm1 = pd.DataFrame(confusion_matrix(y_test, y_test))
sn.heatmap(cm1,ax=axes[1])
score = accuracy_score(y_test, y4)
score

## ExtraTree 

In [47]:
etc=ExtraTreesClassifier(max_depth=107)
etc_2=ExtraTreesClassifier(max_depth=107)
etc_3=ExtraTreesClassifier(max_depth=107)
etc.fit(X_train, y_train)
etc_2.fit(X_train_2, y_train_2)
etc_3.fit(X_train_3, y_train_3)
etq1= etc.predict(X_train)
eq1 = etc.predict(X_test)
etq2= etc_2.predict(X_train_2)
eq2 = etc_2.predict(X_test_2)
etq3= etc_3.predict(X_train_3)
eq3 = etc_3.predict(X_test_3)

ValueError: Number of features of the model must match the input. Model n_features is 90 and input n_features is 13 

In [ ]:
print('Accuracy via prediction on training set q1: ', accuracy_score(y_train, etq1))
print('Accuracy via prediction on test set q1: ', accuracy_score(y_test, eq1))
print('Accuracy via prediction on training set q2: ', accuracy_score(y_train_2, etq2))
print('Accuracy via prediction on test set q2: ', accuracy_score(y_test_2, eq2))
print('Accuracy via prediction on training set q3: ', accuracy_score(y_train_3, etq3))
print('Accuracy via prediction on test set q3: ', accuracy_score(y_test_3, eq3))

In [ ]:
test3

## ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler

### Q1

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
y_train_model=pd.get_dummies(y_train)

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(103, activation='relu', input_shape=[X_train.shape[1],]))
ann.add(tf.keras.layers.Dense(256, activation='relu'))
ann.add(tf.keras.layers.Dense(128, activation='relu'))
ann.add(tf.keras.layers.Dense(64, activation='relu'))
ann.add(tf.keras.layers.Dense(32, activation='relu'))
ann.add(tf.keras.layers.Dense(22, activation='softmax'))

In [ ]:
ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ann.summary()

In [ ]:
ann.fit(X_train, y_train_model, epochs=100)

In [ ]:
ann.save('q1_model.h5')

In [ ]:
y_pred = ann.predict(X_test)
y_pred_train=ann.predict(X_train)

In [ ]:
test_predictions=[0]*y_pred.shape[0]
for i in range(y_pred.shape[0]):
    test_predictions[i]=np.argmax(y_pred[i])+1
train_predictions=[0]*y_pred_train.shape[0]
for i in range(y_pred_train.shape[0]):
    train_predictions[i]=np.argmax(y_pred_train[i])+1

In [ ]:
print('Accuracy via prediction on test set: ', r2_score(y_test,test_predictions))
print('Accuracy via prediction on test set: ', r2_score(y_train,train_predictions))

In [ ]:
difference=y_test-test_predictions
sns.kdeplot(abs(difference))

### Q2 

In [ ]:
X_train_2 = np.asarray(X_train_2).astype(np.float32)
X_test_2 = np.asarray(X_test_2).astype(np.float32)
sc = StandardScaler()
X_train_2 = sc.fit_transform(X_train_2)
X_test_2 = sc.transform(X_test_2)

In [ ]:
y_train_model=pd.get_dummies(y_train_2)

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(103, activation='relu', input_shape=[X_train_2.shape[1],]))
ann.add(tf.keras.layers.Dense(256, activation='relu'))
ann.add(tf.keras.layers.Dense(128, activation='relu'))
ann.add(tf.keras.layers.Dense(64, activation='relu'))
ann.add(tf.keras.layers.Dense(32, activation='relu'))
ann.add(tf.keras.layers.Dense(20, activation='softmax'))

In [ ]:
ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ann.summary()

In [ ]:
ann.fit(X_train_2, y_train_model, epochs=100)

In [ ]:
ann.save('q2_model.h5')

In [ ]:
y_pred = ann.predict(X_test_2)
y_pred_train=ann.predict(X_train_2)

In [ ]:
test_predictions=[0]*y_pred.shape[0]
for i in range(y_pred.shape[0]):
    test_predictions[i]=np.argmax(y_pred[i])+1
train_predictions=[0]*y_pred_train.shape[0]
for i in range(y_pred_train.shape[0]):
    train_predictions[i]=np.argmax(y_pred_train[i])+1

In [ ]:
print('Accuracy via prediction on test set: ', accuracy_score(y_test_2,test_predictions))
print('Accuracy via prediction on test set: ', accuracy_score(y_train_2,train_predictions))

In [ ]:
difference=y_test_2-test_predictions
sns.kdeplot(abs(difference))

### Q3

In [ ]:
X_train_3 = np.asarray(X_train_3).astype(np.float32)
X_test_3 = np.asarray(X_test_3).astype(np.float32)
sc = StandardScaler()
X_train_3 = sc.fit_transform(X_train_3)
X_test_3 = sc.transform(X_test_3)

In [ ]:
y_train_model=pd.get_dummies(y_train_3)

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(103, activation='relu', input_shape=[X_train_3.shape[1],]))
ann.add(tf.keras.layers.Dense(256, activation='relu'))
ann.add(tf.keras.layers.Dense(128, activation='relu'))
ann.add(tf.keras.layers.Dense(64, activation='relu'))
ann.add(tf.keras.layers.Dense(32, activation='relu'))
ann.add(tf.keras.layers.Dense(18, activation='softmax'))

In [ ]:
ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
ann.summary()

In [ ]:
ann.fit(X_train_3, y_train_model, epochs=100)

In [ ]:
ann.save('q3_model.h5')

In [ ]:
y_pred = ann.predict(X_test_3)
y_pred_train=ann.predict(X_train_3)

In [ ]:
test_predictions=[0]*y_pred.shape[0]
for i in range(y_pred.shape[0]):
    test_predictions[i]=np.argmax(y_pred[i])+1
train_predictions=[0]*y_pred_train.shape[0]
for i in range(y_pred_train.shape[0]):
    train_predictions[i]=np.argmax(y_pred_train[i])+1

In [ ]:
print('Accuracy via prediction on test set: ', r2_score(y_test_3,test_predictions))
print('Accuracy via prediction on test set: ', r2_score(y_train_3,train_predictions))

In [ ]:
difference=y_test_3-test_predictions
sns.kdeplot(abs(difference))

In [ ]:
pred = {
        'yeet':90,
        'keep':'243'}


In [ ]:
def prediction_dataset_genarator(pred):
                pred_dataset = pd.DataFrame(list(pred.values()),columns=pred.keys())
                #pred_data = np.asarray(pred_dataset).astype(np.float32)
                return pred_dataset

In [ ]:
df =prediction_dataset_genarator(pred)
df